### Purpose: Predicting which plant a user has at home

### Use: The goal is to collect data and create a model that can effectively predict which plant the user has when uploading a photo on the platform.

### Problem type: Image Classification

### Data Collection: The data was collected by members of the team and was limited to the available plants in their household and surroundings. So some of the plants are also limited to one stage of its growing, for example the tomato has only pictures of its fruit unripened, so a plant with a red tomato could be misclassified.

### Design Choices: The choice was to use ML.NET due to the easier integration and experience that the team had using Net.Core, and also due to it's easier integration with Azure Functions. Due to this choice, some of the options were limited when building the model, after some research on the available libraries in C# for machine learning, SciSharp.TensorFlow and Sci.Keras were taken into account to build the neural networks but none of them allowed for using a pre-trained model as a base to a customized CNN, due to this reason, there was a limitation to use one of the available DNN pre-trained models in ML.NET which on the other hand only allows pre-trained models, in our case and due to the short amount of data, it was a better choice to use pre-trained models instead of only the CNN without the pre-trained model as a base.

In [2]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.7.0"
#r "nuget:Microsoft.ML.Vision,1.7.0"    
#r "nuget:Microsoft.ML.ImageAnalytics,1.7.0"    
#r "nuget:Microsoft.Extensions.ML,1.7.0"
#r "nuget:SciSharp.TensorFlow.Redist-Windows-GPU,1.14.0"
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

Installed Packages Microsoft.Extensions.ML, 1.7.0 Microsoft.ML, 1.7.0 Microsoft.ML.ImageAnalytics, 1.7.0 Microsoft.ML.Vision, 1.7.0 SciSharp.TensorFlow.Redist-Windows-GPU, 1.14.0 XPlot.Plotly, 2.0.0

In [30]:
using Microsoft.ML.Data;
using Microsoft.ML;
using Microsoft.ML.Vision;
using System.IO;
using System;
using static Microsoft.ML.Transforms.ValueToKeyMappingEstimator;

public static IEnumerable<ImageData> LoadImages(
            string folder,
            bool useFolderNameAsLabel = true)
            => LoadImagesFromDirectory(folder, useFolderNameAsLabel)
                .Select(x => new ImageData(x.imagePath, x.label));

public static IEnumerable<(string imagePath, string label)> LoadImagesFromDirectory(
            string folder,
            bool useFolderNameasLabel)
        {
            var imagesPath = Directory
                .GetFiles(folder, "*", searchOption: SearchOption.AllDirectories)
                .Where(x => Path.GetExtension(x) == ".jpg" || Path.GetExtension(x) == ".png");

            return useFolderNameasLabel
                ? imagesPath.Select(imagePath => (imagePath, Directory.GetParent(imagePath).Name))
                : imagesPath.Select(imagePath =>
                {
                    var label = Path.GetFileName(imagePath);
                    for (var index = 0; index < label.Length; index++)
                    {
                        if (!char.IsLetter(label[index]))
                        {
                            label = label.Substring(0, index);
                            break;
                        }
                    }
                    return (imagePath, label);
  });
}

        public static IEnumerable<ImagePredictionInput> LoadLocalImages(
            string folder,
            bool useFolderNameAsLabel = true)
        {
            return LoadImagesFromDirectory(folder, useFolderNameAsLabel)
                .Select(x => new ImagePredictionInput(
                    image: File.ReadAllBytes(x.imagePath),
                    label: x.label,
                    imageFileName: Path.GetFileName(x.imagePath)));
        }

public class ImagePredictionOutput
    {
        [ColumnName("Score")]
        public float[] Score;

        [ColumnName("PredictedLabel")]
        public string PredictedLabel;
    }

 public class ImageData
{
    public ImageData(string imagePath, string label)
    {
        ImagePath = imagePath;
        Label = label;
    }

    public readonly string ImagePath;
    public readonly string Label;
}

  public class ImagePredictionInput
    {
        public ImagePredictionInput()
        {
        }

        public ImagePredictionInput(byte[] image, string label, string imageFileName)
        {
            Image = image;
            Label = label;
            ImageFileName = imageFileName;
        }

        public byte[] Image {get;set;}

        public string Label {get;set;}

        public string ImageFileName {get;set;}
    }

const string initialDataset = @"C:\Users\luisf\Downloads\Machine Learning personal dataset\Machine Learning personal dataset";
const string initialReducedDataset =  @"C:\Users\luisf\Downloads\Machine Learning personal dataset - Cópia\Machine Learning personal dataset";
const string augmentedDataset = @"C:\Users\luisf\Downloads\Machine Learning personal dataset AUG";
const string finalDataset = @"C:\Users\luisf\Downloads\Machine Learning personal Final\Machine Learning personal dataset";
string imagesFolderPathForPredictions = @"C:\Users\luisf\Downloads\TestImages";

var mlContext = new MLContext(seed: 1);


(56,23): warning CS0649: Field 'ImagePredictionOutput.PredictedLabel' is never assigned to, and will always have its default value null

(53,24): warning CS0649: Field 'ImagePredictionOutput.Score' is never assigned to, and will always have its default value null



----

## First, the initial dataset was tested to understand what prediction could be retrieved from it using the pre-trained model ResNetv50

In [4]:
// 1. Load the initial dataset and shuffle so it'll be better balanced before splitting into test and train data
IEnumerable<ImageData> images = LoadImages(folder: initialDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// 2. Load Images with in-memory type within the IDataView and Transform Labels to Keys (Categorical)
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // 3. Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

// 5. Define the model's training pipeline using DNN default values
var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Console.WriteLine($"Micro accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics.MacroAccuracy}");
           

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\irvtqicw.xfq\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 
Micro accuracy: 0,47161572052401746
Macro accuracy: 0,33066534914361


In [5]:
display(metrics);

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
2.714478799505919,0.3257601780618385,0.33066534914361,0.47161572052401746,0,0,<null>,"[ 0.36630819344600707, 4.528228435419033, 0, 2.592148128720884, 0, 4.100341547018465, 1.2663614071903921, 0, 0, 10.641962688541167, 0, 0, 0.9268349505539969, 0, 3.2894549154731973, 0, 10.569197476002808, 0, 0, 0 ... (155 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], PerClassRecall: [ 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], Counts: [ [ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ] ... (155 more) ], NumberOfClasses: 175 }"


----

## After testing with the first model, it is possible to see that a low accuracy is achieved, possibly because the model was trained with about 120 label where the images were below 5, which means that such amount of images could result in overfitting, for example, one flower had only two images and both in a vase, when trying to predict this flower , if an image had no vase it would fail to predict this was the actual flower or present a lower accuracy than if the plant was in a vase. The macro accuracy being way lower than the micro accuracy, shows that there is some class imbalance, which proves that some labels are doing much better than others.

## The decision was to reduce the dataset and keep the labels where there is at least 5 images of a plant, reducing it from 181 labels to about 61 labels.

In [37]:
// 1. Load the initial dataset and shuffle so it'll be better balanced before splitting into test and train data
IEnumerable<ImageData> images = LoadImages(folder: initialReducedDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// 2. Load Images with in-memory type within the IDataView and Transform Labels to Keys (Categorical)
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // 3. Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

// 5. Define the model's training pipeline using DNN default values
var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Console.WriteLine($"Micro accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics.MacroAccuracy}");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\1mqpfest.rrg\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 
Micro accuracy: 0,6502732240437158
Macro accuracy: 0,5703781512605043


index,value
0,3.238132265194406
1,4.0605745509352
2,1.308571013285548
3,0.3771600899516253
4,5.223123405987968
5,0
6,1.0811994062989954
7,0.5204162247699701
8,1.2803244885324023
9,0.5213234873802339


---- 

### Tuning hyperparameters to enhance the model

### Pre-trained Model choice fell upon MobilenetV2 since it reaches a compromise between performance and accuracy, in this case, it presented a higher accuracy than ResNet50 v2 as it can seen below: (ResNet was used so far, since it is the standard on ML.Net)
### MobilenetV2 is less heavy in size and also performs faster, about half the time for both training and predicting. 
###
### For the learning rate, it 0.01 seemed to have the best prediction, after both 0.001 and 0.0001 were tested.

In [69]:
IEnumerable<ImageData> images = LoadImages(folder: initialReducedDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var options = new ImageClassificationTrainer.Options()
 {
    FeatureColumnName = "Image",
    LabelColumnName = "LabelAsKey",
    Arch = ImageClassificationTrainer.Architecture.MobilenetV2,
    Epoch = 50,
    BatchSize = 10,
    LearningRate = 0.01f,
    ValidationSet = testDataView
};

var pipeline = mlContext.MulticlassClassification.Trainers.ImageClassification(options)
        .Append(mlContext.Transforms.Conversion.MapKeyToValue(
           outputColumnName: "PredictedLabel",
           inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\4ytctbdq.yhr\custom_retrained_model_based_on_mobilenet_v2.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [70]:
metrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
1.3204091038030943,0.6471624758191168,0.6116045991045991,0.6502732240437158,0,0,<null>,"[ 2.5275069864081727, 0.8507972442513009, 1.2587559498908567, 1.5499836006139227, 3.1261535857901226, 2.9538180398395193, 0.030014577043898632, 1.6689439377463802, 0.902478338447415, 0.5524437782751567, 2.2175381840295634, 0.6018216122627691, 0.28967138299145356, 2.8135954177138167, 4.169473036898637, 1.384722738162065, 1.0032072390254, 2.557472886171664, 0.044248192791129036, 0.03055836011814575 ... (38 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.5, 1, 0.6666666666666666, 0, 0, 0, 0.5, 0.75, 1, 0.5, 0.5, 1, 0.75, 0, 0, 1, 1, 0.6, 0.25, 1 ... (38 more) ], PerClassRecall: [ 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0, 0, 0, 1, 0.42857142857142855, 0.5, 0.8, 0.5, 1, 1, 0, 0, 0.5, 1, 0.5, 1, 1 ... (38 more) ], Counts: [ [ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1 ... (38 more) ] ... (38 more) ], NumberOfClasses: 58 }"


### The accuracy is still low, since the images are not that varied in some classes, so the next step was to process the data inside the dataset and multiply the images, such as rotating them and decreasing the quality to improve the accuracy when different images are evaluated. This step was performed in the project DataManipulation that contains the methods to perform this, consisting of flipping, rotating, mirroring and adjusting the quality of the image.

### Then the new model is trained with the chosen pre-trained model but also with the default one (ResNetv2 50) to compare the difference between them. It is possible to see that the one using MobilenetV2 has a slightly better accuracy and a lower log loss.
### The data manipulation proved to be efficient, raising the accuracy from about 65% to around 99,5%.

In [56]:
IEnumerable<ImageData> images = LoadImages(folder: finalDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// 2. Load Images with in-memory type within the IDataView and Transform Labels to Keys (Categorical)
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: finalDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // 3. Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var options = new ImageClassificationTrainer.Options()
 {
    FeatureColumnName = "Image",
    LabelColumnName = "LabelAsKey",
            //    // Just by changing/selecting InceptionV3/MobilenetV2/ResnetV250  
            //    // you can try a different DNN architecture (TensorFlow pre-trained model). 
    Arch = ImageClassificationTrainer.Architecture.MobilenetV2,
    Epoch = 50,       //100
    BatchSize = 10,
    LearningRate = 0.01f,
    ValidationSet = testDataView
};

var pipeline = mlContext.MulticlassClassification.Trainers.ImageClassification(options)
        .Append(mlContext.Transforms.Conversion.MapKeyToValue(
           outputColumnName: "PredictedLabel",
           inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics2 = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\dlyjh0f1.4ak\custom_retrained_model_based_on_mobilenet_v2.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [57]:
metrics2

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
0.02533325123197426,0.9932780171973637,0.9951044832460498,0.9950604667007324,0,0,<null>,"[ 0.013258341214067193, 0.0029608738444084053, 0.01603074655223816, 0.19819676579263876, 0.005447281342060925, 0.015169074557539287, 0.014376796551478688, 0.03477905469984159, 0.024819933444580267, 0.009501605624579738, 0.011995665834515199, 0.0363841114269482, 0.003824555556132884, 0.011507205831813238, 0.026757017690566993, 0.10143298578098622, 0.12291497848504465, 0.010639888846718777, 0.023519932449300052, 0.031135604426496155 ... (41 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 1, 1, 1, 1, 1, 1, 1, 0.9652173913043478, 0.9720930232558139, 1, 0.9917127071823204, 1, 1, 1, 1, 1, 1, 1, 1, 1 ... (41 more) ], PerClassRecall: [ 1, 1, 1, 0.9444444444444444, 1, 1, 1, 0.9823008849557522, 1, 1, 1, 1, 1, 1, 1, 1, 0.925, 1, 0.9913793103448276, 1 ... (41 more) ], Counts: [ [ 95, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 119, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 209, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 359, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 172, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 37, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 115, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60 ... (41 more) ] ... (41 more) ], NumberOfClasses: 61 }"


In [17]:
IEnumerable<ImageData> images = LoadImages(folder: augmentedDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// 2. Load Images with in-memory type within the IDataView and Transform Labels to Keys (Categorical)
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: augmentedDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // 3. Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

// 5. Define the model's training pipeline using DNN default values
var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics1 = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\h53sdtxo.04e\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [19]:
Console.WriteLine($"Micro accuracy: {metrics1.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics1.MacroAccuracy}");
Console.WriteLine($"Log loss: {metrics1.LogLoss}");
Console.WriteLine($"Log loss reduction: {metrics1.LogLossReduction}");

Micro accuracy: 0,9925976696367375
Macro accuracy: 0,9937590517188771
Log loss: 0,04162477905448925
Log loss reduction: 0,9906119985674973


In [35]:
IEnumerable<ImageData> images = LoadImages(folder: finalDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// 2. Load Images with in-memory type within the IDataView and Transform Labels to Keys (Categorical)
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: finalDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // 3. Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

// 5. Define the model's training pipeline using DNN default values
var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\fivuy5ik.gpp\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [36]:
Console.WriteLine($"Micro accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics.MacroAccuracy}");
Console.WriteLine($"Log loss: {metrics.LogLoss}");
Console.WriteLine($"Log loss reduction: {metrics.LogLossReduction}");

Micro accuracy: 0,9921648782149549
Macro accuracy: 0,9913303593965694
Log loss: 0,03925245924054842
Log loss reduction: 0,9895491547966805
